# Sweep attempt

In [1]:
!pip install wandb

In [2]:
import wandb
import numpy as np
import sys
import torch

In [4]:
from funcs import regression_system
from funcs import fcnn
from funcs import dataset

In [3]:
wandb.login()

wandb: Currently logged in as: amf2288 (fagerheim). Use `wandb login --relogin` to force relogin


True

In [ ]:
sweep_config = {
    'method': 'bayes',
    'name': 'sweep',
    'early_terminate':
        'type':'hyperband',
        'min_iter':3}

In [ ]:
metric = {
    'name': 'loss',      #what's the name of loss here? it's definted in regression_system right?
    'goal': 'minimize'}

In [6]:
'''seed=123
batch_size=256
input_channels=len(inputs)
output_channels=1
conv_layers = 2
kernel = 5
kernel_hidden = 3
activation="ReLU"
arch="fcnn"
epochs=100
save_path=BASE
save_name="test-3.pt"
lr=0.0025
wd=0.023133758465751404''';

In [ ]:
parameters_dict ={
    'batch_size': {
        'values': [16,32,64,128,256,512,1024]},
    'conv_layers': {
        'values': [1,2,3,4,5]},
    'kernal': {
        'values': [2,3,4,5,6]}
    'kernal_hidden': {
        'values': [2,3,4,5,6]}
    'activation': {
        'values': ['ReLU', 'LeakyReLu', 'Tanh']}}

In [ ]:
parameters_dict.update({
    'seed': {
        'value': 123},
    'epochs': {
        'value': 100},
    'input_channels': {
        'value': len(inputs)},
    'output_channels': {
        'value': 1},
    'arch': {
        'value': 'fcnn'}})

In [ ]:
parameters_dict.update({
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1}
    'wd': {
        'distribution': 'uniform',
        'min': 0,
        'max': 1}})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = 'first_sweep')

In [ ]:
wandb.agent(sweep_id)